# Background

Sebagai data scientist, tugas utama kita akan selalu bergelut dengan data. Namun, selama ini, apakah kita sadar darimana data kita berasal? bagaimana data kita dikumpulkan? dsb dsb? Untuk menjadi seorang data scientist yang baik, saya rasa pengetahuan dan pengalmaan dalam mengumpulkan data cukup penting. Apakah anda pernah kesulitan dalam mencari data? apakah anda pernah kesulitan dalam melakukan pelabelan data? apakah kalian percaya bahwa data seharusnya gratis dan mudah untuk diakses. It's Data for Democracy!

Untuk membuat data dapat diakses oleh semua orang, pihak penyedia jasa juga harus memberikan batasan guna mengurangi penggunaan berlebihan dan penyalahgunaan data oleh pihak-pihak yang tidak bertanggungjawab. Untuk menanggulangi permasalahn ini, kita bisa memanfaatkan teknologi API (Application Program Interface). Dengan API, kita dapat mengatur bagaimana data kita dapat diakses secara umum.

Dalam Capstone kali ini, kita akan mencoba membuat API menggunakan python + flask agar data kita dapat diakses secara umum. Secara konsep, kita akan membangun aplikasi python menggunakan flask yang dapat mengatur, membaa, dan mengirimkan response terhadap request user.


**Data yang digunakan:** 
- books_c.csv

**Environtments:**
- python 
- pandas
- flask 
- gunicorn


**Goals**
1. Berhasil membuat Flask APP yang berfungsi sebagai API yang memberikan data dalam format JSON
2. Berhasil membuat minimal 2 endpoint statis (atau lebih) dan 1 endpoint dinamis(atau lebih) menggunakan routing
3. Berhasil melakukan deployment Flask APP ke Heroku

*Notes: menggunakan endpoints yang sudah dicontohkan tidak akan dihitung sebagai endpoint hasil kerja capstone*

# Membangun API Python dalam 6 menit 
*Disclaimer: Course ini adalah course singkat untuk memperkenalkan student kedalam dunia **backend**. Akan ada sangat banyak kekurangan dari konsep API yang ada saat ini. Untuk kemudahan, kita hanya akan membahas konsep dasar dari API dan dan mewujudkannya dalam bentuk Flask App

Kita akan mencoba membangun Flask App sebagai API, oleh karena itu jika belum memiliki library `Flask`, silakan install menggunakan `pip install flask`. Berikut adalah beberapa library yang akan kita butuhkan. Cobalah import library tersebut sebelum menginstallnya. 

In [28]:
# !pip install flask
# !pip install pandas
# !pip install requests
# !pip install gunicorn

In [29]:
import flask

In [30]:
import pandas as pd

In [31]:
import requests

In [32]:
import gunicorn

In [47]:
books = pd.read_csv('data/books_c.csv')
books.head(5)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.56,0439785960,9780439785969,eng,652,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.49,0439358078,9780439358071,eng,870,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling,4.47,0439554934,9780439554930,eng,320,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,4.55,043965548X,9780439655484,eng,435,2149872,33964


In [34]:
pd.crosstab(
    index=books['title'],
    columns='average_rating').sort_values(by='average_rating', ascending=False)

col_0,average_rating
title,
'Salem's Lot,11
One Hundred Years of Solitude,11
The Brothers Karamazov,10
The Iliad,10
A Midsummer Night's Dream,9
...,...
Insects & Spiders,1
Inside Himalaya,1
Inside Job,1


In [45]:
books_melt = books.reset_index().melt(id_vars=['title'])
books_melt[books_melt['title']=='Inside Job']

,title,variable,value
8007,Inside Job,index,8007
21721,Inside Job,bookID,24978
35435,Inside Job,authors,Connie Willis
49149,Inside Job,average_rating,3.66
62863,Inside Job,isbn,1596060247
76577,Inside Job,isbn13,9781596060241
90291,Inside Job,language_code,eng
104005,Inside Job,# num_pages,99
117719,Inside Job,ratings_count,1398
131433,Inside Job,text_reviews_count,176


In [55]:
books.unstack()

bookID              0           1
                    1           2
                    2           3
                    3           4
                    4           5
                             ... 
text_reviews_count  13709    1060
                    13710     361
                    13711    1485
                    13712      38
                    13713      33
Length: 137140, dtype: object

In [49]:
pulsar = pd.read_csv('data/pulsar_stars.csv')
pulsar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17898 entries, 0 to 17897
Data columns (total 9 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0    Mean of the integrated profile                17898 non-null  float64
 1    Standard deviation of the integrated profile  17898 non-null  float64
 2    Excess kurtosis of the integrated profile     17898 non-null  float64
 3    Skewness of the integrated profile            17898 non-null  float64
 4    Mean of the DM-SNR curve                      17898 non-null  float64
 5    Standard deviation of the DM-SNR curve        17898 non-null  float64
 6    Excess kurtosis of the DM-SNR curve           17898 non-null  float64
 7    Skewness of the DM-SNR curve                  17898 non-null  float64
 8   target_class                                   17898 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 1.2 MB


In [50]:
pulsar.head(3)

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0


In [53]:
pulsar.stack().groupby('target_class').max()#.reset_index().groupby('Symbols').mean()

KeyError: 'target_class'

Membuat aplikasi Flask sesederhana menulis code dalam 7 baris !
cobalah buat file `app.py` yang berisi script berikut :

```python
from flask import Flask, request 
app = Flask(__name__) 

@app.route('/home')
def home():
    return 'Hello World'

if __name__ == '__main__':
    app.run(debug=True, port=5000)
```

Setelah membuat file `app.py` berisikan script diatas, cobalah jalankan file tersebut melalui terminal dengan masuk ke direktori yang sama dengan file tersebut, dan jalankan sintaks `python app.py`. Setelah menjalankan sintaks tersebut, maka aplikasi kita akan berjalan dengan alamat `localhost` dengan port `5000`, atau dapat kalian akses melalui browser dengan alamat `localhost:5000`. 

pada baris ke-4, terdapat sintaks `@app.route('/home')`, yang berarti bahwa fungsi atau method dibawahnya (fungsi `home`) akan dijalankan ketika user mengakses rute atau alaman `/home`. Sekarang, cobalah akses `localhost:5000/home`, maka kalian akan mendapatkan tulisan 'Hello World'

Pada baris terakhir, `debug=True` akan menampilkan semua log aktivitas dari app kita dalam terminal/comman prompt/shell dimana app kita dijalankan. 

Dari contoh script diatas, kita dapat katakan bahwa rute atau `route` adalah endpoints, atau tempat dimana sistem dan user saling bersentuhan. Dalam kasus diatas, untuk mendapatkan 'Hello World', maka kita harus mengakses endpoint `/home`. Sekali lagi, **representasi endpoint adalah route**.

## Query Arguments

Setelah mengetahui kerangka utama dalam membuat aplikasi Flask, cukup penting untuk mengetahui Query Arguments. Ketika kita melakukan pencarian pada mesin pencari seperti google.com, jika kita perhatikan pada alamat (url) yang diberikan, terdapat query arguments yang diberikan setelah `google.com/` yang diawali dengan `?`. 

Format query pada url pada ummnya adalah `example.com?arg1=value1&arg2=value2`, dimana query akan diawali dengan `?`, dihubungkan dengan `&`, dan setiap query adalah pasangan `key` dan `value`

```python
from flask import Flask, request 
app = Flask(__name__) 

@app.route('/query')
def query_example():
    key1 = 'name'
    key2 = 'age'
    name = request.args[key1] # Jika key tidak disertakan dalam URL, maka akan terjadi server error
    age = request.args.get(key2) # Jika key tidak disertakan dalam URL, maka age akan bernilai None
    return(f"Hello, {name}, yoau are {age} years old")

if __name__ == '__main__':
    app.run(debug=True, port=5000) 
```

Silakan update file `app.py` dengan script diatas, lalu akses `localhost:5000/query?name=Budi Setiawan`. Perhatikan error yang ditampilkan. Sekarang, cobalah akses  `localhost:5000/query?name=Budi Setiawan&age=32`. Lalu bandingkan ketika kita mengakses  `localhost:5000/query?age=32`. Dari contoh diatas, terdapat dua cara untuk mendaptkan arguments dari url yang dikirim oleh user. Pertama adalah dengan menggunakan `request.args[key]`. Namun cara ini cukup beresiko mengingat jika key tidak tersedia dalam url, maka akan terjadi server error. Cara pertama ini jarang digunakna kecuali jika kita memang mewajibkan adanya argumen tertentu. Cara yang kedua adalah `request.args.get(key)`. Cara kedua ini lebih sering digunakan karena secara otomatis akan memberikan nilai None jika key tidak terdapat dalam url.

Dalam API, penggunaan query arguments dapat sangat membantu terutama dalam endpoint dinamis. Kita akan pelajari bagaimana query arguments akan sangat membantu ktia dalam menyederhanakan pembuatan API server kita. 

## API Methods

Senada dengan konsep **CRUD** (**C**reate, **R**read, **U**pdate, **D**elete) dalam database, dalam API konsep tersebut diadopsi menjadi 
- POST(CREATE), 
- GET(READ), 
- PUT(UPDATE), 
- DELETE

Secara konsep, method POST, GET, PUT, dan DELETE dibuat untuk mengatur API agar dapat lebih terstruktur. Meskipun pada prakterknya, endpoint yang kita definisikan menggunakan method "POST" tidaklah harus untuk membuat data. Artinya, tidak ada aturan yang mengharuskan kita untuk menuliskan jenis method dalam setiap endpoints kita (default endpoints menggunakan method GET). Namun, untuk tujuan kerapihan dan mengikuti kaidah yang benar, kita akan coba mendefinisikan endpoints kita dengan method tertentu, dan melihat manfaat dari penggunaan jenis method dalam setiap endpoint. 



Berikut adalah contoh sederhana mengenai method POST dan GET
```python 
from flask import Flask, request 
app = Flask(__name__)

@app.route('/coba', methods=['GET', 'POST'])
def terserah():
    if request.method == 'GET':
        return "Ini adalah Hasil method GET"
    else:
        return "Ini adalah hasil method POST"


if __name__ == '__main__':
    app.run(debug=True, port=5000) 
```

Dan berikut adalah contoh yang lebih rumit
```python
from flask import Flask, request 
app = Flask(__name__)

@app.route('/form', methods=['GET', 'POST']) #allow both GET and POST requests
def form():
    if request.method == 'POST':  # Hanya akan tampil setelah melakukan POST (submit) form
        key1 = 'name'
        key2 = 'age'
        name = request.form.get(key1)
        age = request.form[key2]
        
        return (f'''<h1>Your Name  is: {name}</h1>
                   <h1>Your Age is: {age}</h1>
                ''')
   
    
    return '''<form method="POST">
                  Name: <input type="text" name="name"><br>
                  Age: <input type="text" name="age"><br>
                  <input type="submit" value="Submit"><br>
              </form>'''

if __name__ == '__main__':
    app.run(debug=True, port=5000) 
```

Silakan update file `app.py` kalian dengan sintaks diatas, lalu akses `localhost:5000/form`, aka akan muncul form sederhana dengan tombol "submit". Mengapa form tampil terlebih dahulu? karena dalam url `localhost:5000/form`, kita tidak mendefinisikan method kita sebagai POST atau GET. Maka, secara default request kita akan dianggap sebagai GET. Apakah kita bisa menentukan method kita dalam url? Kami sarankan untuk tidak mencobanya. 

Dalam sintaks diatas, kita mendefinisikan rute atau endpoint `/form` sebagai GET atau POST. Hal ini berarti endpoint tersebut dapat menerima request GET atau POST. 

Setelah kita mengisi form dan menekan tombol "submit", maka kita akan melakukan request POST dengan mengirimkan data-data berupa nama dan umur kepada server kita. Oleh sebab itu, server kita akan menjalankan perintah yang berada dalam kondisi `if request.method == 'POST':`

Cobalah untuk mengganti `if request.method == 'POST':` menjadi `if request.method == 'GET':`, maka kalian akan melihat perbedannya. 

Jika kalian penasaran, cobalah ubah `@app.route('/form', methods=['GET', 'POST'])` menjadi `@app.route('/form', methods=['POST'])`


## JSON Data Type

JSON adalah standar tipe data dalam pengiriman data dengan protokol HTTP. Pada dasarnya, JSON (JavaScript Object Name) sangat mirip, atau bisa dibilang identik dengan tipe data `dictionary` dalam python. Dictionary terdiri dari pasangan `key`:`value`. Berikut adalah contoh bentuk dan akses data dictionary. 

In [ ]:
books = {'author': 'Anthony',
         'num_of_page': 123,
         'year' : 2019
        }
print(f"Pengaran buku: {books['author']}, dengan halaman {books['num_of_page']}, diterbitkan pada {books['year']}")

Kita akan mencoba mengirimkan data JSON menggunakan method POST ke endpoint kita. Dikarenakan browser biasa tidak bisa menangani hal tersebut, kita bisa gunakan [postman](https://www.getpostman.com/) atau jupyter notebook untuk membantu.
Silakan update file `app.py` dengan script berikut: 

```python
from flask import Flask, request 
app = Flask(__name__) 

@app.route('/json', methods=['POST']) 
def json_exmp():
    
    req = request.get_json(force=True) # melakukan parsing data json, menyimpannya sebagai dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')

if __name__ == '__main__':
    app.run(debug=True, port=5000) #run app in debug mode on port 5000
```

Setelah server tersebut diupdate, kita akan coba melakukan request menggunakan library requests pada python

In [ ]:
import requests

url = 'http://localhost:5000/json'
data = {"name" : "Andi", 
        "age" : 10,
        "address" : "Jakarta"
       }
r = requests.post(url,json=data) # pastikan gunakan requests.pos, bukan requests.get
print(r)

Response 200 berarti API kita berhasil mengirimkan atau melakukan request yang kita kirim. Untuk mempelajari jenis-jenis repsonse yang dikirimkan melalui HTTP, silakan merujuk ke [sumber ini](https://restfulapi.net/http-status-codes/). Objek yang kita kirimkan berupa `requests.models.Response`. Untuk mendapatkan isi dari respon tersebut, kita dapat gunakan `.text` atau `.content`

In [ ]:
print(type(r))

In [ ]:
print(r.content)
print(r.text)

## Data Fetching

Kembali ke tujuan utama, kita akan mencoba membuat API yang mampu mengirimkan data dalam bentuk DataFrame (dalam format JSON). Dalam hal ini, kita akan coba menggunakan data `books_c.csv` yang terdapat dalam folder `data_input`. Dalam contoh kali ini, kita akan mengaplikasikan dynamic routing. Konsepnya adalah kita tidak secara eksplisit membuat route atau endpoint, melainkan menyimpannya dalam variabel. Hal ini akan sangat berguna jika data yang ingin kita berikan sangat beragam dan bergantung pada keinginan user.

Untuk melakukan dynamic routing, cukup simpan variabel kedalam tanda `< >`, lalu masukkan variabel tersebut sebagai parameter fungsi di endpoint yang bersangkutan. 

```python 
from flask import Flask, request 
import pandas as pd 
app = Flask(__name__) 

# mendapatkan keseluruhan data dari <data_name>
@app.route('/data/get/<data_name>', methods=['GET']) 
def get_data(data_name): 
    data = pd.read_csv('data/' + str(data_name))
    return (data.to_json())
    

# mendapatkan data dengan filter nilai <value> pada kolom <column>
@app.route('/data/get/equal/<data_name>/<column>/<value>', methods=['GET']) 
def get_data_equal(data_name, column, value): 
    data = pd.read_csv('data/' + str(data_name))
    mask = data[column] == value
    data = data[mask]
    return (data.to_json())

if __name__ == '__main__':
    app.run(debug=True, port=5000) 
```

Untuk meminta data menggunakan API yang telah ada, silakan jalankan cells berikut

In [ ]:
import requests
import pandas as pd

In [ ]:
url1 = 'http://localhost:5000/data/get/books_c.csv'
r = requests.get(url1)
r_pd = pd.DataFrame(r.json())

In [ ]:
r_pd.head()

In [ ]:
url2 = 'http://localhost:5000/data/get/equal/books_c.csv/isbn/0439785960'
r = requests.get(url2)
r_pd = pd.DataFrame(r.json())
r_pd.head()

In [ ]:
url3 = 'http://localhost:5000/data/get/equal/books_c.csv/authors/J.K. Rowling'
r = requests.get(url3)
r_pd = pd.DataFrame(r.json())
r_pd.tail()

Dari contoh diatas, kita sudah dapat melakukan fetching data menggunakan API dan seberapa bermanfaatnya melakukan dynamic routing. Kedepannya, diharapkan pengetahuan ini dapat dikembangkan dengan melakukan fetching data dari database. 

## Deploying Flask APP 

Sekarang, setelah Flask App kita sudah berhasil dijalankan, kita akan coba deploy app tersebut **(beserta environmentnya)** ke app hosting/web hosting/cloud. Untuk saat ini, kita akan menggunakan [heroku](https://www.heroku.com/) karena **gratis**. Untuk melakukan deployment, silakan registrasi akun baru. 

Langkah ini dapat kalian ikuti pada [sumber ini](https://stackabuse.com/deploying-a-flask-application-to-heroku/)

### Get the requirements

Silakan buat environment baru, lalu hanya install `pandas`, `flask`, dan `gunicorn`. Setelah libary tersebut berhasil diinstal, jalankan `pip freeze > requirements.txt` untuk mengeksport semua library kedalam file `requirements.txt`. Perlu diingat bahwa Heroku hanya bisa mengenali libary yang kita pakai jika kita menuliskannya kedalam requirements.txt

### Create Heroku App
Untuk membuat heroku app, cukup klik new->app pada dashboard heroku. Perlu diingat bahwa nama app di heroku haruslah unik. 

### Deployment

Setelah kita membuat dan menamai app kita di heroku, kita harus mengupload file app kita dari lokal. Untuk melakukan hal ini, kita akan menggunakan git. Cara termudah adalah dengan membuat repository baru di github, lalu sambungkan aplikasi heroku degan repository github tersebut. 
![](res/deploy-1.PNG)

Nyalakan `Automatic Deploy` jika ingin melakukan deployment ulang setiap terjadi perubahan pada repositoy secara otomatis (kurang disarankan). Setelah repository terhubung, klik tombol deploy branch yang berada di bagian bawah app heroku (jika tidak ada branch pada repository github, secara default branch master yang akan dipilih). 

![](res/deploy-2.PNG)

# Test Your API Endpoints

Setelah deployment app kita behasil, kita bisa mencoba mengaksesnya melalui browser, atau jupyter notebook(lebih disarankan). Mari coba beberapa endpoints yang telah kita coba lakukan di lokal. 

Untuk catatan, tidak perlu menuliskan port pada url heroku, karena kita sudah mengaturnya pada file `Procfile`

In [ ]:
import requests
import pandas as pd 
heroku_url = 'https://algo-capstone.herokuapp.com/data/get/books_c.csv'
r = requests.get(heroku_url)
r_pd = pd.DataFrame(r.json())
r_pd

# Document your API

Hal terpenting dalam pembuatan API adalah dokumentasi. Aplikasi yang bagus harus memiliki dokumentasi untuk user yang belum tahu cara menggunakannya. Hal ini juga memudahkan tim kami untuk melakukan penilaian. Oleh karena itu, buatlah dokumentasi API sederhana dengan format : 
- Deskripsi API 
- Endpoints
___
```
Deskripsi: (Contoh Dokumentasi)
API ini dinominasikan sebagai capstone project yang berguna untuk mengirimkan data kepada user. Proses wrangling dilakukan sesuai endpoint-endpoint yang dimaksud. Base url dari aplikasi ini adalah https://algo-capstone.herokuapp.com

Endpoints: 
    1. / , method = GET
    Merupakan endpoint home, dan akan mengembalikan nilai berupa string selamat datang
    
    2. /data/get/<data_name> , method = GET
    Mengembalikan data <data_name> dalam bentuk JSON. Beberapa data yang tersedia adalah : 
        - books_c.csv
        - pulsar_stars.csv (tidak digunakan, hanya contoh saja)
        
    3. /data/get/equal/<data_name>/<column>/<value>
    Mengembalikan <data_name> yang telah di filter dimana nilai pada <column> = <value>

```
___

# Tips 

Jika dirasa ada kesulitan untuk menentukan informasi apa yang akan dikirimkan dalam sebuah endpoint (membuat endpoint), cobalah beralih ke sudut pandang user. Sebagai user, informasi apa yang ingin kita dapatkan dari data tersebut? Akan ada sangat banyak kemungkinan yang bisa diambil. Kalian bisa melakukannya! 

Happy Coding !
~ Team Algoritma